### Noise shift at the 3 million mark

Building on the work of @rkaveland [here](https://www.kaggle.com/kaaveland/tps202112-ctgan-artifacts-chunks?kernelSessionId=82663352) and @ambrosm [here](https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/292381)

The Wilderness_Area features are noisy. One expects every sample to be in exactly one Wilderness Area, but in the data we see samples that have 0, 1, or more areas assigned. 

I noticed the noise model shifts right around the 3 million sample mark in the train data, and seems to remain consistent throughout the test data.

Define Noise as the sum of the assigned Wilderness Areas minus one. Then plot the cumulative sum. It's pretty dramatic.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv(
         '/kaggle/input/tabular-playground-series-dec-2021/train.csv')
del train['Cover_Type']

test = pd.read_csv(
         '/kaggle/input/tabular-playground-series-dec-2021/test.csv')

traintest = pd.concat([train, test], axis=0)

wa_sum = (
    traintest['Wilderness_Area1'] * 1 +
    traintest['Wilderness_Area2'] * 1 +
    traintest['Wilderness_Area3'] * 1 +
    traintest['Wilderness_Area4'] * 1).values

plt.plot(np.cumsum(wa_sum - 1))
plt.show()


Plotting the individual noise levels.

In [ ]:
plt.plot(np.cumsum(wa_sum == 0))
plt.plot(np.cumsum(wa_sum == 2))
plt.plot(np.cumsum(wa_sum > 2))

plt.show()

### Implications

I'm not sure of the impact. 
- It probably explains why adding the sum of the Wilderness Area features as a feature to your model is helpful. 
- Your CV scores using the entire training set probably aren't going to be representative of the test score. 
- You may actually be better off training on only the final million samples. 
- Perhaps adding some transformation of Id as a feature will improve your model.
- Tree learners cannot extrapolate, but neural nets can. Extrapolation is essential, as the noise values in test set are beyond the values in training data.